# 타겟 ETF(11개) <-> 매크로 변수(12개, df3 제외) 간 그레인저 인과 검정

    검정 횟수 : 11 x 12 x 2(양방향) = 264 번.
    실 검정 횟수는 정상성 만족하지 않는 데이터 제외하였므로 더 적음.



---
---
---

# 1. xlsx 파일 파싱하여 매크로변수 추출

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader.data as web
from statsmodels.tsa.stattools import adfuller, grangercausalitytests

### 중요도1.xlsx 파일 파싱하여 매크로변수 추출

In [2]:
# Excel 파일 경로
file_path = './1.xlsx'

# Excel 파일 읽기
xl = pd.ExcelFile(file_path)

# 주별 데이터 시트에서 데이터프레임 생성
df1 = xl.parse('주별 데이터')
df1.columns = ['date', 'initial_jobless_claim']
df1['date'] = pd.to_datetime(df1['date'])
df1.set_index('date', inplace = True)

# 일별 데이터 시트에서 데이터프레임 생성
df2 = xl.parse('일별 데이터')
df2.columns = ['date', 'gdp_now_forecast']
df2['date'] = pd.to_datetime(df2['date'])
df2.set_index('date', inplace = True)

# # 결과 확인
# print("Initial Jobless Claims (Weekly):")
# print(df1.head())

# print("\nGDP Now Forecast (Daily):")
# print(df2.head())

### 중요도2.xlsx 파일 파싱하여 매크로변수 추출

In [3]:
# Excel 파일 경로
file_path = './2.xlsx'

# Excel 파일 읽기
xl = pd.ExcelFile(file_path)

# 분기별 데이터에서 실질 GDP 연율 전기대비 추출
df3 = xl.parse('분기별 데이터')
df3.columns = ['date', 'real_gdp_qoq']
df3['date'] = pd.to_datetime(df3['date'])
df3.set_index('date', inplace = True)

# 월별 데이터에서 ISM 제조업지수, ISM 비제조업지수, CPI 추출
monthly_data = xl.parse('월별 데이터')
monthly_data['날짜'] = pd.to_datetime(monthly_data['날짜'], format = '%Y/%m')
monthly_data.set_index('날짜', inplace = True)

df4 = monthly_data[['[미국]ISM제조업지수']].rename(columns = {'[미국]ISM제조업지수': 'ism_manufacturing'})
df5 = monthly_data[['[미국]ISM비제조업지수']].rename(columns = {'[미국]ISM비제조업지수': 'ism_nonmanufacturing'})
df6 = monthly_data[['[미국]CPI(원지수)']].rename(columns = {'[미국]CPI(원지수)': 'cpi'})

# 일별 데이터에서 BEI 기대 인플레이션 추출
df7 = xl.parse('일별 데이터')
df7.columns = ['date', 'bei_breakeven_inflation']
df7['date'] = pd.to_datetime(df7['date'])
df7.set_index('date', inplace = True)

# WTI 유가 추출
df8 = xl.parse('WTI')
df8.columns = ['date', 'wti_oil_price']
df8['date'] = pd.to_datetime(df8['date'])
df8.set_index('date', inplace = True)

# 금값 추출
df9 = xl.parse('금')
df9.columns = ['date', 'gold_price']
df9['date'] = pd.to_datetime(df9['date'])
df9.set_index('date', inplace = True)

# # 결과 확인
# print("Real GDP QoQ (Quarterly):")
# print(df3.head())

# print("\nISM Manufacturing (Monthly):")
# print(df4.head())

# print("\nISM Non-Manufacturing (Monthly):")
# print(df5.head())

# print("\nCPI (Monthly):")
# print(df6.head())

# print("\nBEI Breakeven Inflation (Daily):")
# print(df7.head())

# print("\nWTI Oil Price (Daily):")
# print(df8.head())

# print("\nGold Price (Daily):")
# print(df9.head())

### 중요도3.xlsx 파일 파싱하여 매크로변수 추출

In [4]:
# Excel 파일 경로
file_path = './3.xlsx'

# Excel 파일 읽기
xl = pd.ExcelFile(file_path)

# 월별데이터 시트에서 데이터프레임 생성
monthly_data = xl.parse('월별데이터')
monthly_data['날짜'] = pd.to_datetime(monthly_data['날짜'], format = '%Y/%m')
monthly_data.set_index('날짜', inplace = True)

# 각 매크로 변수별 데이터프레임 생성
df10 = monthly_data[['[미국]실업률[미국_SA]']].rename(columns = {'[미국]실업률[미국_SA]': 'unemployment_rate'})
df11 = monthly_data[['[미국]PCE 가격지수 전년대비']].rename(columns = {'[미국]PCE 가격지수 전년대비': 'pce_price_index_yoy'})
df12 = monthly_data[['[미국]Core PCE 가격지수 전년대비']].rename(columns = {'[미국]Core PCE 가격지수 전년대비': 'core_pce_price_index_yoy'})
df13 = monthly_data[['[미국]비농업부문고용자수']].rename(columns = {'[미국]비농업부문고용자수': 'nonfarm_payroll'})

# # 결과 확인
# print("US Unemployment Rate (Monthly):")
# print(df10.head())

# print("\nPCE Price Index YoY (Monthly):")
# print(df11.head())

# print("\nCore PCE Price Index YoY (Monthly):")
# print(df12.head())

# print("\nNonfarm Payrolls (Monthly):")
# print(df13.head())

- df1 : 미국 신규 실업수당청구건수 (initial_jobless_claim)
- df2 : 미국 GDP Now GDP Forecast (gdp_now_forecast)
- df3 : 실질 GDP 연율 전기대비 (real_gdp_qoq)
- df4 : ISM 제조업지수 (ism_manufacturing)
- df5 : ISM 비제조업지수 (ism_nonmanufacturing)
- df6 : CPI 원지수 (cpi)
- df7 : BEI 기대 인플레이션 (bei_breakeven_inflation)
- df8 : WTI 유가 (wti_oil_price)
- df9 : 금값 (gold_price)
- df10 : 미국 실업률 (unemployment_rate)
- df11 : PCE 가격 지수 전년 대비 (pce_price_index_yoy)
- df12 : Core PCE 가격 지수 전년 대비 (core_pce_price_index_yoy)
- df13 : 비농업부문 고용자수 (nonfarm_payroll)

# 2. 일별 / 주별 / 분기별 데이터(매크로 변수)를 월말 데이터로 리샘플링, 결측치 채워넣기

In [5]:
def daily_to_monthly_end(df):
    # 인덱스가 날짜인지 확인
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # 월의 마지막 날짜 선택
    monthly_end = df.resample('M').last()

    return monthly_end

def weekly_to_monthly_end(df):
    # 인덱스가 날짜인지 확인
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # 각 데이터 포인트에 해당 월의 마지막 날짜 할당
    df['month_end'] = df.index + pd.offsets.MonthEnd(0)

    # 월말 기준으로 그룹화하고 마지막 주의 데이터 선택
    monthly_end = df.groupby('month_end').last()

    return monthly_end

def quarterly_to_monthly_end(df):
    # 인덱스가 날짜인지 확인
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # 월말 날짜로 리샘플링 (forward fill 사용)
    monthly_end = df.resample('M').ffill()

    return monthly_end

In [6]:
# 일별 데이터를 월말 데이터로 리샘플링
df2 = daily_to_monthly_end(df2)
df7 = daily_to_monthly_end(df7)
df8 = daily_to_monthly_end(df8)
df9 = daily_to_monthly_end(df9)

# 주별 데이터를 월말 데이터로 리샘플링
df1 = weekly_to_monthly_end(df1)

# 분기 데이터를 월말 데이터로 리샘플링
df3 = quarterly_to_monthly_end(df3)

# 결측치 채우기
for df in [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]:
    df.fillna(method='bfill', inplace = True)  # 뒤의 값으로 결측치 채우기
    df.fillna(method='ffill', inplace = True)  # 앞의 값으로 남은 결측치 채우기

# # 결과 확인
# for i, df in enumerate([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13], start=1):
#     print(f"df{i} after filling NaN values:")
#     print(df.head())
#     print(f"Number of remaining NaN values in df{i}: {df.isna().sum().sum()}")
#     print("\n")

<ipython-input-6-306f833de3b2>:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace = True)  # 뒤의 값으로 결측치 채우기
<ipython-input-6-306f833de3b2>:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace = True)  # 앞의 값으로 남은 결측치 채우기


# 3. 타겟 ETF(11개) 별 데이터프레임 확인

In [7]:
# ETF 데이터 불러오기 함수
def get_monthly_returns(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start = start_date, end = end_date, interval = '1mo')

    x = stock_data['Close'].resample('M').last().pct_change().dropna() # 조정 종가(Adj Close) -> 일반종가(Close)로 변경

    x.name = 'Return' # = 수익률

    return x

In [8]:
ticker_list = ['IWM', 'SPY', 'VTV', 'VUG', 'MTUM', 'QUAL',
               'VYMI', 'USMV', 'KBE', 'IYK', 'IYC']

start_date = '1940-01-01'
end_date = '2024-09-19'

# 4. 타겟 ETF별(11개), 각 매크로 변수 별(13개) 그레인저 인과성 검정 진행

- ETF 하나씩 불러오고, 각 df1, df2, ..., df13 매크로 변수 데이터와 날짜 인덱스 매칭시킴
- ETF <-> df1, ETF <-> df2, ..., ETF <-> df13 양방향 그레인저 인과성 검정 수행

In [9]:
def adf_test(series, signif=0.05):
    """ADF 테스트를 통해 정상성을 확인"""
    result = adfuller(series.dropna(), autolag='AIC')
    p_value = result[1]
    if p_value < signif:
        return True  # 정상성 있음
    else:
        return False  # 정상성 없음

def granger_causality(data, x_column, y_column, max_lag=4):
    """x_column이 y_column에 그레인저 인과성이 있는지 확인"""
    result = grangercausalitytests(data[[y_column, x_column]], max_lag, verbose=False)
    # 가장 낮은 p-value 추출
    p_values = [round(test[0]['ssr_ftest'][1], 4) for test in result.values()]
    return min(p_values)

# 매크로 변수와 ETF 데이터 로드
# (df1, df2, ..., df13 : 12개의 매크로 변수(df3 보류), ticker_list : 11개의 ETF 리스트)
macro_dfs = [df1, df2, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]
macro_names = [
    'initial_jobless_claim',
    'gdp_now_forecast',
    'ism_manufacturing',
    'ism_nonmanufacturing',
    'cpi',
    'bei_breakeven_inflation',
    'wti_oil_price',
    'gold_price',
    'unemployment_rate',
    'pce_price_index_yoy',
    'core_pce_price_index_yoy',
    'nonfarm_payroll'
]

# 결과를 저장할 리스트
results = []

# 각 ETF에 대해 처리
for ticker in ticker_list:
    etf_data = get_monthly_returns(ticker, start_date, end_date)

    # ETF 정상성 검정 (ADF 테스트)
    if not adf_test(etf_data):
        print(f"ETF {ticker} 데이터가 정상성을 만족하지 않습니다.")
        continue

    # 각 매크로 변수에 대해 처리
    for df, factor_name in zip(macro_dfs, macro_names):
        # 데이터 정렬 및 결측치 제거
        etf_data = etf_data[~etf_data.index.duplicated(keep='first')]  # 중복 인덱스 제거
        df = df[~df.index.duplicated(keep='first')]  # 중복 인덱스 제거
        merged_data = pd.concat([etf_data, df], axis=1, join='inner').dropna()  # 열 단위 병합(axis=1), 같은 인덱스 기준 데이터 JOIN

        if not merged_data.empty:
            etf_column = merged_data.columns[0]  # ETF 수익률 (Return)
            factor_column = merged_data.columns[1]  # 매크로 변수 (각 매크로 변수명)

            # 매크로 변수의 정상성 검정 (ADF 테스트)
            if not adf_test(merged_data[factor_column]):
                print(f"매크로 변수 {factor_name}가 정상성을 만족하지 않습니다.")
                continue

            # 그레인저 인과성 검정
            p_value_x_to_y = granger_causality(merged_data, factor_column, etf_column)
            p_value_y_to_x = granger_causality(merged_data, etf_column, factor_column)

            # 그레인저 인과성 여부 판단 및 결과 처리
            if p_value_x_to_y < 0.05 and p_value_y_to_x >= 0.05:
                granger_causality_result = 'X -> Y'
            elif p_value_y_to_x < 0.05 and p_value_x_to_y >= 0.05:
                granger_causality_result = 'Y -> X'
            elif p_value_x_to_y < 0.05 and p_value_y_to_x < 0.05:
                granger_causality_result = 'X <-> Y'
            else:
                granger_causality_result = 'FALSE'

            # 결과 저장
            results.append({
                'ETF': ticker,
                'Factor': factor_name,
                'p-value (X->Y)': p_value_x_to_y,
                'p-value (Y->X)': p_value_y_to_x,
                'Granger Causality': granger_causality_result
            })

        else:
            print(f"No overlapping data for {ticker} and {factor_name}")

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# CSV 파일로 저장
results_df.to_csv('granger_causality_results_with_adf.csv', index=False)

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: Future

매크로 변수 wti_oil_price가 정상성을 만족하지 않습니다.
매크로 변수 gold_price가 정상성을 만족하지 않습니다.
매크로 변수 core_pce_price_index_yoy가 정상성을 만족하지 않습니다.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul

매크로 변수 wti_oil_price가 정상성을 만족하지 않습니다.
매크로 변수 gold_price가 정상성을 만족하지 않습니다.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul

...

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: Future

매크로 변수 wti_oil_price가 정상성을 만족하지 않습니다.
매크로 변수 gold_price가 정상성을 만족하지 않습니다.
매크로 변수 core_pce_price_index_yoy가 정상성을 만족하지 않습니다.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul

매크로 변수 wti_oil_price가 정상성을 만족하지 않습니다.
매크로 변수 gold_price가 정상성을 만족하지 않습니다.
매크로 변수 core_pce_price_index_yoy가 정상성을 만족하지 않습니다.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
